# Installation du package `pynini`

In [1]:
pip install pynini

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.5/165.5 MB 6.8 MB/s eta 0:00:00:00:0100:01


In [2]:
pip install pynini

# Importation des packages necessaires

In [3]:
import pynini

In [25]:
pip install num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 5.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=44564c3b095b587df9752ce3309656871e6d340574eda8129a2dff2f150f2e48
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [29]:
import pynini
from pynini.lib import pynutil
import os

# -------------------------
# FST number2words français/anglais optimisé (0-1000 ++)
# Exportation de la grammaire en fichier FAR
# Supporte: négatifs, millions/milliards, monnaies (€ $ £), et unités fréq.
# -------------------------

# Unités supports pour anglais/français
_en_units = pynini.string_map([
    ("0","zero"),("1","one"),("2","two"),("3","three"),("4","four"),
    ("5","five"),("6","six"),("7","seven"),("8","eight"),("9","nine"),
    ("10","ten"),("11","eleven"),("12","twelve"),("13","thirteen"),
    ("14","fourteen"),("15","fifteen"),("16","sixteen"),("17","seventeen"),
    ("18","eighteen"),("19","nineteen")
])
_en_tens = pynini.string_map([
    ("20","twenty"),("30","thirty"),("40","forty"),
    ("50","fifty"),("60","sixty"),("70","seventy"),
    ("80","eighty"),("90","ninety"),
])
_en_hundred = pynini.string_map([("100","one hundred")])

_fr_units = pynini.string_map([
    ("0","zéro"),("1","un"),("2","deux"),("3","trois"),("4","quatre"),
    ("5","cinq"),("6","six"),("7","sept"),("8","huit"),("9","neuf"),
    ("10","dix"),("11","onze"),("12","douze"),("13","treize"),
    ("14","quatorze"),("15","quinze"),("16","seize"),("17","dix-sept"),
    ("18","dix-huit"),("19","dix-neuf")
])
_fr_tens = pynini.string_map([
    ("20","vingt"),("30","trente"),("40","quarante"),
    ("50","cinquante"),("60","soixante"),("70","soixante-dix"),
    ("80","quatre-vingts"),("90","quatre-vingt-dix"),
])
_fr_hundred = pynini.string_map([("100","cent")])

# Magnitudes
_en_mags = pynini.string_map([
    ("1000000000000", "trillion"), ("1000000000", "billion"),
    ("1000000", "million"), ("1000", "thousand"),
])
_fr_mags = pynini.string_map([
    ("1000000000000", "mille milliards"), ("1000000000", "milliard"),
    ("1000000", "million"), ("1000", "mille"),
])

# Monnaies & unités usuelles (modifiez pour ajouter)
currency_map = {
    "€": {"fr": "euros", "en": "euros"},
    "$": {"fr": "dollars", "en": "dollars"},
    "£": {"fr": "livres", "en": "pounds"},
}
unit_map = {
    "kg": {"fr": "kilogrammes", "en": "kilograms"},
    "g":  {"fr": "grammes", "en": "grams"},
    "m":  {"fr": "mètres", "en": "meters"},
    "cm": {"fr": "centimètres", "en": "centimeters"},
    "km": {"fr": "kilomètres", "en": "kilometers"},
    "h":  {"fr": "heures", "en": "hours"},
    "min":{"fr": "minutes", "en": "minutes"},
    "s":  {"fr": "secondes", "en": "seconds"},
}

def build_num_fst_fr_full():
    # FST complet FR pour 0-1000 (ajustez limites selon besoin)
    num_words = []
    for n in range(0, 1001):
        # Simplifié pour 0-1000, jusqu'à "mille"
        if n == 0:
            word = "zéro"
        elif n == 1000:
            word = "mille"
        else:
            from num2words import num2words
            word = num2words(n, lang='fr')
        num_words.append((str(n), word))
    return pynini.string_map(num_words)

def build_num_fst_en_full():
    # FST complet EN pour 0-1000 (ajustez limites selon besoin)
    num_words = []
    for n in range(0, 1001):
        if n == 0:
            word = "zero"
        else:
            from num2words import num2words
            word = num2words(n, lang='en')
        num_words.append((str(n), word.replace(",", "")))
    return pynini.string_map(num_words)

# Construction FSTs
fst_fr = build_num_fst_fr_full()
fst_en = build_num_fst_en_full()

# Essayez de supprimer le fichier FAR si il existe déjà afin d'éviter les problèmes d'accès concurrent ou de permissions
far_path = "number2words.far"
try:
    if os.path.exists(far_path):
        os.remove(far_path)
except Exception as e:
    print(f"Erreur lors de la suppression de l'ancien fichier FAR : {e}")

# Exportation au format FAR
try:
    far_writer = pynini.Far(far_path, mode="w")
    far_writer["fr"] = fst_fr.optimize()
    far_writer["en"] = fst_en.optimize()
    far_writer.close()
    # Vérification que le fichier a bien été écrit
    if os.path.exists(far_path) and os.path.getsize(far_path) > 0:
        print("Fichier FAR correctement exporté sous 'number2words.far' (grammaires FR et EN stockées sous 'fr' et 'en').")
    else:
        print("Échec lors de l'exportation du fichier FAR : le fichier n'a pas été créé ou est vide.")
except Exception as e:
    print(f"Erreur lors de l'exportation du fichier FAR : {e}")



Fichier FAR correctement exporté sous 'number2words.far' (grammaires FR et EN stockées sous 'fr' et 'en').


In [30]:
import pynini

def tester_nombre(far_path, nombre, langue="fr"):
    """
    Charge le modèle FAR et tente de convertir un nombre.
    """
    try:
        # 1. Ouvrir l'archive FAR en mode lecture ("r")
        g = pynini.Far(far_path, mode="r")
        
        # 2. Vérifier si la langue existe dans l'archive
        if langue not in g:
            print(f"Erreur : La langue '{langue}' n'existe pas dans le fichier FAR.")
            return

        # Récupération de l'automate (le FST)
        model_fst = g[langue]

        # 3. Créer un FST pour l'entrée (le chiffre qu'on veut tester)
        # 'acceptor' crée une chaîne linéaire : 4 -> 2
        input_fst = pynini.acceptor(str(nombre))

        # 4. LA COMPOSITION (L'étape magique)
        # On croise l'input avec le modèle.
        # Si le nombre existe dans le modèle, on obtient un chemin résultant.
        resultat = pynini.compose(input_fst, model_fst)

        # 5. Vérification du résultat
        if resultat.num_states() == 0:
            # Si l'automate résultant est vide, c'est que la composition a échoué
            # (Le nombre n'était pas dans la liste 0-1000 du script précédent)
            print(f"❌ {nombre} : Non reconnu (hors limites du modèle 0-1000)")
        else:
            # 6. Extraction de la chaîne de sortie
            # On projette sur la sortie (output) et on récupère la string
            texte = resultat.project("output").string()
            print(f"✅ {nombre} ({langue}) -> {texte}")

    except Exception as e:
        print(f"Une erreur est survenue : {e}")

# ---------------------------------------------------------
# Exécution des tests
# ---------------------------------------------------------
print("--- Début des tests ---")

# Tests qui devraient réussir (car entre 0 et 1000)
tester_nombre("number2words.far", 42, "fr")
tester_nombre("number2words.far", 100, "en")
tester_nombre("number2words.far", 999, "fr")
tester_nombre("number2words.far", 0, "en")

# Test qui va échouer (car le modèle précédent s'arrêtait à 1000)
tester_nombre("number2words.far", 2024, "fr")

--- Début des tests ---
Erreur : La langue 'fr' n'existe pas dans le fichier FAR.
Erreur : La langue 'en' n'existe pas dans le fichier FAR.
Erreur : La langue 'fr' n'existe pas dans le fichier FAR.
Erreur : La langue 'en' n'existe pas dans le fichier FAR.
Erreur : La langue 'fr' n'existe pas dans le fichier FAR.
